In [ ]:
import psycopg2
from psycopg2 import OperationalError
import requests

In [ ]:
def create_table(
    tabname: str,
    dbname: str,
    user: str,
    password: str,
    host: int,
    port: int
) -> None:
    valid_tab_names = ['demand', 'weather']
    assert tabname in valid_tab_names, "Enter a valid table name."
    
    conn_params = {
        'dbname': dbname,
        'user': user,
        'password': password,
        'host': host,
        'port': port
    }
    
    # Connect to your postgres DB
    conn = psycopg2.connect(**conn_params)

    cur = conn.cursor()

    demand_cols = '''
        period VARCHAR(100),
        subba VARCHAR(100),
        subba_name VARCHAR(100),	
        parent VARCHAR(100),	
        parent_name VARCHAR(100),	
        timezone VARCHAR(100),	
        value INTEGER,	
        value_units VARCHAR(100)
    '''

    weather_cols = '''
        time VARCHAR(100),
        temperature_2m_max	REAL,
        temperature_2m_min	REAL,
        temperature_2m_mean REAL
    '''

    tab_schema = {
        'demand': demand_cols,
        'weather': weather_cols
    }

    # Define the CREATE TABLE statement
    create_table_query = f'''
    CREATE TABLE {tabname} (
        id SERIAL PRIMARY KEY, 
        {tab_schema.get(tabname)}
    );
    '''
    
    # Execute the query
    cur.execute(create_table_query)

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

In [ ]:
params = {
        'tabname': 'weather',
        'dbname': 'db_demand',
        'user': 'dbuser',
        'password': '123',
        'host': 'localhost',
        'port': '5432'
    }
    
create_table(**params)

In [ ]:
def generate_url(
    start_date, 
    end_date, 
    offset, 
    chunk_len, 
    api_key,
    use_api
):
    url_demand = f'https://api.eia.gov/v2/electricity/rto/' +\
    f'daily-region-sub-ba-data/data/?frequency=daily&data[0]=value' + \
    f'&facets[subba][]=ZONJ&start={start_date}&end={end_date}' + \
    f'&sort[0][column]=period&sort[0][direction]=desc&offset={offset}' + \
    f'&length={chunk_len}&api_key={api_key}'

    url_weather_historical = f'https://archive-api.open-meteo.com/v1' + \
    f'/archive?latitude=52.52&longitude=13.41&start_date={start_date}' + \
    f'&end_date={end_date}&daily=weathercode,temperature_2m_max,' + \
    f'temperature_2m_min,temperature_2m_mean' + \
    f'&timezone=America%2FNew_York'
    
    url_weather_latest = f'https://api.open-meteo.com/v1/forecast?' + \
    f'latitude=52.52&longitude=13.41&hourly=temperature_2m&daily=' + \
    f'temperature_2m_max,temperature_2m_min&timezone=America%2F' + \
    f'New_York&past_days=7'

    api_urls = {
        'eia': url_demand,
        'weather_hist': url_weather_historical,
        'weather_latest': url_weather_latest
    }

    return api_urls.get(use_api, None)

In [ ]:
start_date = '2018-01-01'
end_date = '2022-07-31'
offset = 0
chunk_len = 10000
api_key = ''
use_api = 'weather_latest'

url = generate_url(start_date, end_date, offset, chunk_len, api_key, use_api)
url

In [ ]:
def get_data(url):
    response = requests.get(url)

    if response.status_code == 200:  # Success
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [ ]:
def main():
    
    return